In [ ]:
# 🔑 Configuration
GITHUB_TOKEN = ""  # ⚠️ Remplace par ton vrai token

HEADERS = {
    "Accept": "application/vnd.github+json",
    "Authorization": f"Bearer {GITHUB_TOKEN}",
    "X-GitHub-Api-Version": "2022-11-28"
}

import requests
import time
import json
import base64
import os

# Créer le dossier pour les avatars
os.makedirs("avatars", exist_ok=True)

# ------------------------------------------------------------
# UTILITAIRE : Générer des requêtes GitHub alignées sur les queries Twitter
# ------------------------------------------------------------
def get_github_search_queries():
    keywords = [
        "data scientist", "data engineer", "machine learning", "artificial intelligence",
        "deep learning", "data analyst", "big data", "business intelligence",
        "mlops", "nlp", "computer vision", "llm", "genai", "robotics",
        "intelligence artificielle", "analyste données", "scientifique données"
    ]
    locations = ["Morocco", "Maroc", "Casablanca", "Rabat"]
    
    queries = []
    for kw in keywords:
        queries.append(f'"{kw}" in:bio location:"Morocco"')
        queries.append(f'"{kw}" in:bio location:"Maroc"')
        queries.append(f'language:python location:"Morocco" {kw}')
    return queries

# ------------------------------------------------------------
# Télécharger l’image de profil (avatar)
# ------------------------------------------------------------
def download_avatar(avatar_url, username):
    """Télécharge l’avatar et retourne le chemin local."""
    if not avatar_url:
        return None
    
    try:
        ext = "jpg"  # GitHub sert souvent du JPEG/PNG sans extension
        filepath = f"avatars/{username}_avatar.{ext}"
        
        img_resp = requests.get(avatar_url, timeout=10)
        if img_resp.status_code == 200:
            with open(filepath, "wb") as f:
                f.write(img_resp.content)
            return filepath
        else:
            print(f"⚠️ Échec téléchargement avatar pour {username} (code {img_resp.status_code})")
    except Exception as e:
        print(f"⚠️ Erreur téléchargement avatar {username}: {e}")
    return None

# ------------------------------------------------------------
# Récupérer le README du profil
# ------------------------------------------------------------
def fetch_profile_readme(username):
    url = f"https://api.github.com/repos/{username}/{username}/readme"  # ✅ URL corrigée
    resp = requests.get(url, headers=HEADERS)
    if resp.status_code == 200:
        try:
            content_b64 = resp.json().get("content", "")
            return base64.b64decode(content_b64).decode("utf-8")
        except Exception as e:
            print(f"⚠️ Erreur décodage README pour {username}: {e}")
    return ""

# ------------------------------------------------------------
# Collecte complète d’un profil GitHub
# ------------------------------------------------------------
def collect_github_profile(username):
    print(f"📥 Collecte du profil GitHub : {username}")
    
    # 1. Données utilisateur
    user_url = f"https://api.github.com/users/{username}"  # ✅ URL corrigée
    user_resp = requests.get(user_url, headers=HEADERS)
    if user_resp.status_code != 200:
        raise Exception(f"Utilisateur {username} introuvable (code {user_resp.status_code})")
    user_data = user_resp.json()
    
    # 2. Repos publics
    repos_url = f"https://api.github.com/users/{username}/repos"  # ✅ URL corrigée
    repos_resp = requests.get(repos_url, headers=HEADERS, params={"per_page": 100, "sort": "updated"})
    repos = repos_resp.json() if repos_resp.status_code == 200 else []
    
    repo_names = [r["name"] for r in repos if r.get("name")]
    repo_descriptions = [r["description"] for r in repos if r.get("description")]
    
    # 3. README du profil
    profile_readme = fetch_profile_readme(username)
    
    # 4. Télécharger l’avatar
    avatar_local_path = download_avatar(user_data.get("avatar_url"), username)
    
    # ------------------------------------------------------------
    # PROFIL UNIFIÉ – conforme au cahier des charges (Étape 1)
    # Inclut maintenant l’image de profil pour matching visuel (Étape 3)
    # ------------------------------------------------------------
    return {
        "platform": "github",
        "user_id": str(user_data.get("id")),
        "username": user_data.get("login"),
        "name": user_data.get("name") or user_data.get("login"),
        "bio": user_data.get("bio") or "",
        "profile_readme": profile_readme,
        "location": user_data.get("location") or "",
        "email": user_data.get("email") or "",
        "company": user_data.get("company") or "",
        "website": user_data.get("blog") or "",
        "avatar_url": user_data.get("avatar_url"),
        "avatar_local_path": avatar_local_path,  # ✅ Chemin local de l’image
        
        # Activités
        "public_repos_count": user_data.get("public_repos", 0),
        "followers": user_data.get("followers", 0),
        "following": user_data.get("following", 0),
        "repo_names": repo_names,
        "repo_descriptions": [d for d in repo_descriptions if d],
        "recent_commit_messages": [],  # Tu peux l’activer si besoin
        
        # Métadonnées
        "profile_url": user_data.get("html_url"),
        "created_at": user_data.get("created_at"),
        "updated_at": user_data.get("updated_at")
    }

# ------------------------------------------------------------
# Recherche + collecte massive
# ------------------------------------------------------------
def search_and_collect_github_profiles(query, max_users=30):
    print(f"🔍 Recherche GitHub : '{query}'")
    users = []
    page = 1
    
    while len(users) < max_users:
        resp = requests.get(
            "https://api.github.com/search/users",  # ✅ URL corrigée
            headers=HEADERS,
            params={"q": query, "per_page": 100, "page": page}
        )
        if resp.status_code != 200:
            print(f"⚠️ Erreur API ({resp.status_code}) sur la requête : {query}")
            break
        
        data = resp.json()
        if "items" not in data:
            break
        
        for item in data["items"]:
            if len(users) >= max_users:
                break
            if item["login"] not in users:
                users.append(item["login"])
        
        if len(data["items"]) < 100:
            break
        page += 1
        time.sleep(0.35)
    
    profiles = {}
    for i, username in enumerate(users, 1):
        try:
            profile = collect_github_profile(username)
            profiles[username] = profile
            print(f"[{i}/{len(users)}] ✅ {username}")
            time.sleep(0.5)
        except Exception as e:
            print(f"[{i}/{len(users)}] ❌ {username} – {e}")
    
    return profiles

# ------------------------------------------------------------
# LANCEMENT PRINCIPAL
# ------------------------------------------------------------
if __name__ == "__main__":
    github_queries = get_github_search_queries()
    all_profiles = {}
    total_target = 1000
    
    for q in github_queries:
        if len(all_profiles) >= total_target:
            break
        print(f"\n🚀 Lancement requête : {q}")
        remaining = total_target - len(all_profiles)
        batch = search_and_collect_github_profiles(q, max_users=min(20, remaining))
        all_profiles.update(batch)
        time.sleep(1)
    
    # Sauvegarde
    with open("github_unified_profiles.json", "w", encoding="utf-8") as f:
        json.dump(all_profiles, f, indent=4, ensure_ascii=False)
    
    print(f"\n✅ {len(all_profiles)} profils GitHub collectés.")
    print("📁 Profils sauvegardés dans 'github_unified_profiles.json'")
    print("🖼️ Avatars enregistrés dans le dossier 'avatars/'")


🚀 Lancement requête : "data scientist" in:bio location:"Morocco"
🔍 Recherche GitHub : '"data scientist" in:bio location:"Morocco"'
📥 Collecte du profil GitHub : omarmhaimdat
[1/20] ✅ omarmhaimdat
📥 Collecte du profil GitHub : AnasAito
[2/20] ✅ AnasAito
📥 Collecte du profil GitHub : mnijadi
[3/20] ✅ mnijadi
📥 Collecte du profil GitHub : aelghezzaz
[4/20] ✅ aelghezzaz
📥 Collecte du profil GitHub : elreyfahad
[5/20] ✅ elreyfahad
📥 Collecte du profil GitHub : mrdahdah
[6/20] ✅ mrdahdah
📥 Collecte du profil GitHub : brahimbellahcen
[7/20] ✅ brahimbellahcen
📥 Collecte du profil GitHub : mohamedprojects
[8/20] ✅ mohamedprojects
📥 Collecte du profil GitHub : souidiyassine
[9/20] ✅ souidiyassine
📥 Collecte du profil GitHub : AGMach7
[10/20] ✅ AGMach7
📥 Collecte du profil GitHub : tatimohammed
[11/20] ✅ tatimohammed
📥 Collecte du profil GitHub : oujbih
[12/20] ✅ oujbih
📥 Collecte du profil GitHub : salmaelbarbori
[13/20] ✅ salmaelbarbori
📥 Collecte du profil GitHub : abdelkhalek-haddany
[14/20]

In [ ]:
# 🔑 Configuration
GITHUB_TOKEN = ""  # ⚠️ Remplace par ton vrai token

HEADERS = {
    "Accept": "application/vnd.github+json",
    "Authorization": f"Bearer {GITHUB_TOKEN}",
    "X-GitHub-Api-Version": "2022-11-28"
}

import requests
import time
import json
import base64
import os

# Créer le dossier pour les avatars
os.makedirs("avatars", exist_ok=True)

# ------------------------------------------------------------
# UTILITAIRE : Générer des requêtes GitHub alignées sur les queries Twitter
# ------------------------------------------------------------
def get_github_search_queries():
    keywords = [
        "data scientist", "data engineer", "machine learning", "artificial intelligence",
        "deep learning", "data analyst", "big data", "business intelligence",
        "mlops", "nlp", "computer vision", "llm", "genai", "robotics",
        "intelligence artificielle", "analyste données", "scientifique données"
    ]
    locations = ["Morocco", "Maroc", "Casablanca", "Rabat"]
    
    queries = []
    for kw in keywords:
        queries.append(f'"{kw}" in:bio location:"Morocco"')
        queries.append(f'"{kw}" in:bio location:"Maroc"')
        queries.append(f'language:python location:"Morocco" {kw}')
    return queries

# ------------------------------------------------------------
# Télécharger l’image de profil (avatar)
# ------------------------------------------------------------
def download_avatar(avatar_url, username):
    """Télécharge l’avatar et retourne le chemin local."""
    if not avatar_url:
        return None
    
    try:
        ext = "jpg"  # GitHub sert souvent du JPEG/PNG sans extension
        filepath = f"avatars/{username}_avatar.{ext}"
        
        img_resp = requests.get(avatar_url, timeout=10)
        if img_resp.status_code == 200:
            with open(filepath, "wb") as f:
                f.write(img_resp.content)
            return filepath
        else:
            print(f"⚠️ Échec téléchargement avatar pour {username} (code {img_resp.status_code})")
    except Exception as e:
        print(f"⚠️ Erreur téléchargement avatar {username}: {e}")
    return None

# ------------------------------------------------------------
# Récupérer le README du profil
# ------------------------------------------------------------
def fetch_profile_readme(username):
    url = f"https://api.github.com/repos/{username}/{username}/readme"  # ✅ URL corrigée
    resp = requests.get(url, headers=HEADERS)
    if resp.status_code == 200:
        try:
            content_b64 = resp.json().get("content", "")
            return base64.b64decode(content_b64).decode("utf-8")
        except Exception as e:
            print(f"⚠️ Erreur décodage README pour {username}: {e}")
    return ""

# ------------------------------------------------------------
# Collecte complète d’un profil GitHub
# ------------------------------------------------------------
def collect_github_profile(username):
    print(f"📥 Collecte du profil GitHub : {username}")
    
    # 1. Données utilisateur
    user_url = f"https://api.github.com/users/{username}"  # ✅ URL corrigée
    user_resp = requests.get(user_url, headers=HEADERS)
    if user_resp.status_code != 200:
        raise Exception(f"Utilisateur {username} introuvable (code {user_resp.status_code})")
    user_data = user_resp.json()
    
    # 2. Repos publics
    repos_url = f"https://api.github.com/users/{username}/repos"  # ✅ URL corrigée
    repos_resp = requests.get(repos_url, headers=HEADERS, params={"per_page": 100, "sort": "updated"})
    repos = repos_resp.json() if repos_resp.status_code == 200 else []
    
    repo_names = [r["name"] for r in repos if r.get("name")]
    repo_descriptions = [r["description"] for r in repos if r.get("description")]
    
    # 3. README du profil
    profile_readme = fetch_profile_readme(username)
    
    # 4. Télécharger l’avatar
    avatar_local_path = download_avatar(user_data.get("avatar_url"), username)
    
    # ------------------------------------------------------------
    # PROFIL UNIFIÉ – conforme au cahier des charges (Étape 1)
    # Inclut maintenant l’image de profil pour matching visuel (Étape 3)
    # ------------------------------------------------------------
    return {
        "platform": "github",
        "user_id": str(user_data.get("id")),
        "username": user_data.get("login"),
        "name": user_data.get("name") or user_data.get("login"),
        "bio": user_data.get("bio") or "",
        "profile_readme": profile_readme,
        "location": user_data.get("location") or "",
        "email": user_data.get("email") or "",
        "company": user_data.get("company") or "",
        "website": user_data.get("blog") or "",
        "avatar_url": user_data.get("avatar_url"),
        "avatar_local_path": avatar_local_path,  # ✅ Chemin local de l’image
        
        # Activités
        "public_repos_count": user_data.get("public_repos", 0),
        "followers": user_data.get("followers", 0),
        "following": user_data.get("following", 0),
        "repo_names": repo_names,
        "repo_descriptions": [d for d in repo_descriptions if d],
        "recent_commit_messages": [],  # Tu peux l’activer si besoin
        
        # Métadonnées
        "profile_url": user_data.get("html_url"),
        "created_at": user_data.get("created_at"),
        "updated_at": user_data.get("updated_at")
    }

# ------------------------------------------------------------
# Recherche + collecte massive
# ------------------------------------------------------------
def search_and_collect_github_profiles(query, max_users=30):
    print(f"🔍 Recherche GitHub : '{query}'")
    users = []
    page = 1
    
    while len(users) < max_users:
        resp = requests.get(
            "https://api.github.com/search/users",  # ✅ URL corrigée
            headers=HEADERS,
            params={"q": query, "per_page": 100, "page": page}
        )
        if resp.status_code != 200:
            print(f"⚠️ Erreur API ({resp.status_code}) sur la requête : {query}")
            break
        
        data = resp.json()
        if "items" not in data:
            break
        
        for item in data["items"]:
            if len(users) >= max_users:
                break
            if item["login"] not in users:
                users.append(item["login"])
        
        if len(data["items"]) < 100:
            break
        page += 1
        time.sleep(0.35)
    
    profiles = {}
    for i, username in enumerate(users, 1):
        try:
            profile = collect_github_profile(username)
            profiles[username] = profile
            print(f"[{i}/{len(users)}] ✅ {username}")
            time.sleep(0.5)
        except Exception as e:
            print(f"[{i}/{len(users)}] ❌ {username} – {e}")
    
    return profiles

# ------------------------------------------------------------
# LANCEMENT PRINCIPAL
# ------------------------------------------------------------
if __name__ == "__main__":
    github_queries = get_github_search_queries()
    all_profiles = {}
    total_target = 1000
    
    for q in github_queries:
        if len(all_profiles) >= total_target:
            break
        print(f"\n🚀 Lancement requête : {q}")
        remaining = total_target - len(all_profiles)
        batch = search_and_collect_github_profiles(q, max_users=min(20, remaining))
        all_profiles.update(batch)
        time.sleep(1)
    
    # Sauvegarde
    with open("github_unified_profiles.json", "w", encoding="utf-8") as f:
        json.dump(all_profiles, f, indent=4, ensure_ascii=False)
    
    print(f"\n✅ {len(all_profiles)} profils GitHub collectés.")
    print("📁 Profils sauvegardés dans 'github_unified_profiles.json'")
    print("🖼️ Avatars enregistrés dans le dossier 'avatars/'")